## Use CrewAI

- https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/

```
conda create -n crewai
conda activate crewai
sudo apt install python3-pip
pip3 install crewai
pip3 install "crewai[tools]"
pip3 install notebook
```

In [1]:
!pip show api_key_store  # 0.1 (2024-05-27)

Name: api-key-store
Version: 0.1
Summary: API Key Store
Home-page: #
Author: P2P2Learn
Author-email: p2p2learn@outlook.com
License: Apache 2
Location: /home/papagame/Documents/Dad/0-vault/api_keys/apikeystore
Requires: 
Required-by: 


In [2]:
!pip show langchain_community  # 0.0.38 (2024-05-27)

Name: langchain-community
Version: 0.0.38
Summary: Community contributed LangChain integrations.
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/papagame/.local/lib/python3.10/site-packages
Requires: aiohttp, dataclasses-json, langchain-core, langsmith, numpy, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain


In [3]:
! pip show crewai               # 0.30.11 (2024-05-27)

Name: crewai
Version: 0.30.11
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /home/papagame/.local/lib/python3.10/site-packages
Requires: appdirs, click, embedchain, instructor, langchain, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 


In [4]:
! pip show crewai_tools         # 0.2.6 (2024-05-27)

Name: crewai-tools
Version: 0.2.6
Summary: Set of tools for the crewAI framework
Home-page: 
Author: João Moura
Author-email: joaomdmoura@mgail.com
License: 
Location: /home/papagame/.local/lib/python3.10/site-packages
Requires: beautifulsoup4, chromadb, docx2txt, embedchain, lancedb, langchain, openai, pydantic, pyright, pytest, pytube, requests, selenium
Required-by: 


In [5]:
from api_key_store import ApiKeyStore

In [6]:
s = ApiKeyStore()

In [7]:
import os
os.environ["OPENAI_API_KEY"] = s.get_api_key(provider="OPENAI")  # os.environ["API_KEY_OPENAI"]
os.environ["SERPER_API_KEY"] = s.get_api_key(provider="SerpApi")  # os.environ["API_KEY_SERPAPI"]

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'   # gpt-4 was default, throws rate-limit error 

In [8]:
from crewai import Agent
from crewai_tools import SerperDevTool

In [9]:
import crewai   # Version: 0.30.11

In [10]:
search_tool = SerperDevTool()

### Step 1: Assemble Your Agents

In [11]:
# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory="""
    Driven by curiosity, you're at the forefront of innovation, 
    eager to explore and share knowledge that could change the world.
  """,
  tools=[search_tool],
  allow_delegation=True
)

In [12]:
# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory="""
    With a flair for simplifying complex topics, you craft engaging narratives that captivate and educate, 
    bringing new discoveries to light in an accessible manner.
  """,
  tools=[search_tool],
  allow_delegation=False
)

### Step 2: Define the Tasks

In [13]:
from crewai import Task

# Research task
research_task = Task(
  description="""
    Identify the next big trend in {topic}.
    Focus on identifying pros and cons and the overall narrative.
    Your final report should clearly articulate the key points,
    its market opportunities, and potential risks.
  """,
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description="""
    Compose an insightful article on {topic}.
    Focus on the latest trends and how it's impacting the industry.
    This article should be easy to understand, engaging, and positive.
  """,
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

### Step 3: Form the Crew

In [14]:
from crewai import Crew, Process

# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True, # False,  # 
  cache=True, # False,   # 
  max_rpm=100,
  # share_crew=True
)

### Step 4: Kick It Off

In [15]:
from time import time

In [16]:
# Starting the task execution process with enhanced feedback
ts_start = time()
result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
ts_stop = time()
print(result)
print(f"\n\nElapsed time: {ts_stop-ts_start} sec")



> Entering new CrewAgentExecutor chain...
I need to gather information on the latest trends in AI in healthcare, including their pros, cons, market opportunities, and potential risks. I should aim to provide a comprehensive report that clearly articulates the key points for my final answer.

Action:
Search the internet

Action Input:
{
    "search_query": "latest trends in AI in healthcare pros and cons"
}
 


Search results: Title: The Pros and Cons of AI in Healthcare - HITRUST
Link: https://hitrustalliance.net/blog/the-pros-and-cons-of-ai-in-healthcare
Snippet: The future of AI in healthcare hinges on collaboration, innovation, and a strong commitment to responsible AI implementation. However, this ...
---
Title: What is AI in healthcare? Pros, cons & applications - FutureLearn
Link: https://www.futurelearn.com/info/blog/what-is-ai-in-healthcare
Snippet: AI makes healthcare more predictive and proactive by analysing data to improve preventive care recommendations. For more informa

2024-05-27 09:16:25,118 - 124433200467968 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--eea0053cb991e9e0cd71d46bed8f90d2261190c05c5df27d95ecbb4677e6fce0


Thought: I need to research the latest trends and advancements in AI in healthcare to craft an insightful article.

Action: Search the internet

Action Input: {"search_query": "AI in healthcare advancements"}
 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to research the latest trends and advancements in AI in healthcare to craft an insightful article.

Action: Search the internet

Action Input: {"search_query": "latest trends in AI in healthcare"}
 


Search results: Title: Five trends converging for AI-enabled healthcare: Why AI and ... - PwC
Link: https://www.pwc.com/gx/en/industries/healthcare/publications/ai-robotics-new-health/five-trends.html
Snippet: The next decade is moving towards medical solutions – using AI, robotics, and virtual and augmented reality – to deliver intelligent solutions for both evidence ...
---
Title: AI in 2024: Welcome to the 'new normal' in healthcare
Link: https://www.healthca